In [15]:
from sklearn.model_selection import train_test_split
import csv

In [13]:
data = []
labels = []
with open('data/train/train.csv', encoding="utf-8") as df:
    reader = csv.reader(df)
    next(reader)
    for row in reader:
        query_pair = row[3:5]
        data.append(query_pair)
        labels.append(row[5])

In [17]:
print(len(data), len(labels))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)
print(len(X_train), len(X_test))

404290 404290
323432 80858
